In [1]:
# test_param_fixed_income.py
import numpy as np
import pandas as pd
from param_fixed_income import FixedIncomePrmSingle, FixedIncomePrmPort

# ========================================================================
# 1. Generate Input Data
# ========================================================================
def create_sample_yield_curve():
    """Create monthly yield curve DataFrame with 1-360 month maturities"""
    dates = pd.date_range(start="2023-08-29", periods=1)
    columns = np.arange(1, 361)  # 1-360 months
    rates = [5.0 - (i/1000) for i in range(360)]  # Sample rates decreasing from 5%
    return pd.DataFrame([rates], index=dates, columns=columns)

def create_historical_yield_data():
    """Create historical yield data for simulation methods"""
    dates = pd.date_range(start="2023-01-01", periods=100)
    return pd.DataFrame(np.random.uniform(4.0, 5.0, (100, 360)), 
                        index=dates, 
                        columns=np.arange(1, 361))

# Create input data
yield_curve_today = create_sample_yield_curve()
historical_yield_curve = create_historical_yield_data()

# ========================================================================
# 2. Test FixedIncomePrmSingle (Single Bond)
# ========================================================================
print("\n" + "="*60 + "\nTesting Single Bond Pricing and Risk\n" + "="*60)

# ------------------------------------------------------------------------
# Test 1: Zero Coupon Bond (ZCB)
# ------------------------------------------------------------------------
print("\nTest 1: Zero Coupon Bond (ZCB)")

# Initialize ZCB
zcb = FixedIncomePrmSingle(
    yield_curve_today=yield_curve_today,
    maturity=5,  # 5-year maturity
    num_assets=100,
    face_value=1000,
    is_zcb=True
)

# Auto-runs fit() during initialization
print("\n[fit() Results] Calculated bond fundamentals:")
print(f"Price: ${zcb.bond_price:.2f}")
print(f"Duration: {zcb.duration} years")
print(f"Modified Duration: {zcb.modified_duration} years")

# ------------------------------------------------------------------------
# Test 2: DurationNormal Method (Normal VaR/ES)
# ------------------------------------------------------------------------
print("\nTest 2: DurationNormal Method")
print("Purpose: Calculate VaR/ES using duration-based sensitivity analysis")

dn_result = zcb.DurationNormal(
    yield_curve=historical_yield_curve,
    vol="ewma",  # Using EWMA volatility model
    interval=10,  # 10-day risk horizon
    alpha=0.05    # 95% confidence level
)

print("\nResults:")
print(f"10-Day VaR: ${dn_result['var']:,.2f}")
print(f"10-Day ES: ${dn_result['es:']:,.2f}")
print(f"Key Driver: {dn_result['MappedBondPos']:,.2f} (Duration x Position)")

# ------------------------------------------------------------------------
# Test 3: HistoricalSimulation Method
# ------------------------------------------------------------------------
print("\nTest 3: HistoricalSimulation Method")
print("Purpose: Calculate VaR/ES using historical yield changes")

hs_result = zcb.HistoricalSimulation(
    yield_curve=historical_yield_curve,
    interval=10,
    alpha=0.05
)

print("\nResults:")
print(f"10-Day VaR: ${hs_result['var']:,.2f}")
print(f"10-Day ES: ${hs_result['es']:,.2f}")
print(f"New Maturity: {hs_result['NewMaturity']:.2f} years after interval")

# ========================================================================
# 3. Test FixedIncomePrmPort (Bond Portfolio)
# ========================================================================
print("\n" + "="*60 + "\nTesting Bond Portfolio Analysis\n" + "="*60)

# Create mixed portfolio
portfolio = FixedIncomePrmPort(
    yield_curve_today=yield_curve_today,
    maturity_s=np.array([2, 5, 7.5]),          # 2yr ZCB + 5/7.5yr coupon bonds
    num_assets_s=np.array([100, -50, 75]),     # Short position in 2nd bond
    face_value_s=np.array([1000, 1000, 1000]),
    is_zcb_s=np.array([True, False, False]),   # Bond types
    coupon_rate_s=np.array([0, 3.8, 4.2]),     # Coupon rates (ignored for ZCB)
    semi_annual_payment_s=np.array([False, True, True])
)

# ------------------------------------------------------------------------
# Test 4: Portfolio Summary
# ------------------------------------------------------------------------
print("\nTest 4: Portfolio Summary")
print("Purpose: Show portfolio composition and characteristics")

print("\nPortfolio Summary Table:")
print(portfolio.summary[["ZCB", "Maturity", "BondPrice", "Pos", "Duration"]])

# ------------------------------------------------------------------------
# Test 5: Portfolio DurationNormal
# ------------------------------------------------------------------------
print("\nTest 5: Portfolio DurationNormal")
print("Purpose: Portfolio-level duration-based risk analysis")

port_dn = portfolio.DurationNormal(
    yield_curve=historical_yield_curve,
    interval=10,
    alpha=0.05
)

print("\nResults:")
print(f"Portfolio VaR: ${port_dn['var']:,.2f}")
print(f"Portfolio ES: ${port_dn['es']:,.2f}")
print(f"Portfolio Duration: {port_dn['DurationPort']:.2f} years")

# ------------------------------------------------------------------------
# Test 6: Portfolio Historical Simulation
# ------------------------------------------------------------------------
print("\nTest 6: Portfolio Historical Simulation")
print("Purpose: Full portfolio scenario analysis using historical data")

port_hs = portfolio.HistoricalSimulation(
    yield_curve=historical_yield_curve,
    interval=10,
    alpha=0.05
)

print("\nResults:")
print(f"Portfolio VaR: ${port_hs['var']:,.2f}")
print(f"Portfolio ES: ${port_hs['es:']:,.2f}")
print(f"Initial Position Value: ${port_hs['OrigPos']:,.2f}")


Testing Single Bond Pricing and Risk

Test 1: Zero Coupon Bond (ZCB)

[fit() Results] Calculated bond fundamentals:
Price: $785.73
Duration: 5 years
Modified Duration: 4.7646 years

Test 2: DurationNormal Method
Purpose: Calculate VaR/ES using duration-based sensitivity analysis

Results:
10-Day VaR: $7,203.26
10-Day ES: $9,033.18
Key Driver: 374,369.50 (Duration x Position)

Test 3: HistoricalSimulation Method
Purpose: Calculate VaR/ES using historical yield changes

Results:
10-Day VaR: $3,016.89
10-Day ES: $3,374.46
New Maturity: 4.97 years after interval

Testing Bond Portfolio Analysis

Test 4: Portfolio Summary
Purpose: Show portfolio composition and characteristics

Portfolio Summary Table:
     ZCB  Maturity  BondPrice         Pos  Duration
0   True       2.0   907.4270  90742.7000    2.0000
1  False       5.0   952.4515 -47622.5750    4.5888
2  False       7.5   959.1259  71934.4425    6.4863

Test 5: Portfolio DurationNormal
Purpose: Portfolio-level duration-based risk analy